In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import lightgbm as lgb
from gensim.models import Word2Vec

In [2]:
df = pd.read_csv("data1230.csv")
# df = df.drop(["aid", "request_day"], axis=1)
df.head()

,aid,request_day,aid_embed_0,aid_embed_1,aid_embed_2,aid_embed_3,aid_embed_4,aid_embed_5,aid_embed_6,aid_embed_7,...,periods_on_40,periods_on_41,periods_on_42,periods_on_43,periods_on_44,periods_on_45,periods_on_46,periods_on_47,periods_cont,uid
0,357876,17943,0.255362,0.184418,-0.069231,-0.150381,-0.090089,-0.098903,-0.070554,0.014766,...,0,0,0,0,0,0,1,1,2,149292
1,357876,17944,0.255362,0.184418,-0.069231,-0.150381,-0.090089,-0.098903,-0.070554,0.014766,...,0,0,0,0,0,0,1,1,2,142783
2,357876,17945,0.255362,0.184418,-0.069231,-0.150381,-0.090089,-0.098903,-0.070554,0.014766,...,0,0,0,0,0,0,1,1,2,321243
3,357876,17946,0.255362,0.184418,-0.069231,-0.150381,-0.090089,-0.098903,-0.070554,0.014766,...,0,0,0,0,0,0,1,1,2,365753
4,357876,17947,0.255362,0.184418,-0.069231,-0.150381,-0.090089,-0.098903,-0.070554,0.014766,...,0,0,0,0,0,0,1,1,2,366062


In [3]:
# 过滤展示量异常值 只保留95%一下的数据
imp_95 = np.percentile(df.imp, 95)
df = df[df.imp <= imp_95] 

In [23]:
df = df.sort_values(by="request_day")
df.head(20)

,aid,request_day,aid_embed_0,aid_embed_1,aid_embed_2,aid_embed_3,aid_embed_4,aid_embed_5,aid_embed_6,aid_embed_7,...,periods_on_40,periods_on_41,periods_on_42,periods_on_43,periods_on_44,periods_on_45,periods_on_46,periods_on_47,periods_cont,uid
0,357876,17943,0.255362,0.184418,-0.069231,-0.150381,-0.090089,-0.098903,-0.070554,0.014766,...,0,0,0,0,0,0,1,1,2,149292
44580,561658,17943,0.236562,0.142219,-0.098367,-0.028200,-0.120419,0.013902,-0.103383,0.051960,...,1,1,1,1,1,1,1,1,34,1393717
44529,369559,17943,0.370428,1.158772,-0.433347,-1.099412,0.283961,-0.330034,-0.605646,0.030665,...,1,1,1,1,1,1,1,1,34,862386
44525,247140,17943,0.284375,0.018457,0.003244,-0.030531,-0.168585,-0.075440,-0.029333,0.005584,...,1,1,1,1,1,1,1,1,34,548266
44509,559211,17943,0.623272,0.151014,-0.312254,-0.329922,-0.278556,-0.372269,0.089857,0.162009,...,1,1,1,1,1,1,1,1,34,1113841
44496,98215,17943,0.301621,0.223505,-0.140495,-0.026970,-0.112330,0.016428,-0.074869,0.098949,...,1,1,1,1,0,0,0,0,14,949922
2154,305238,17943,0.102438,0.073960,0.034574,-0.011280,-0.050169,-0.119517,-0.100939,0.018450,...,1,1,1,1,1,1,1,1,48,478843
44466,478365,17943,0.413576,0.173307,-0.092753,-0.084096,-0.190765,-0.066491,-0.078280,0.118257,...,1,1,1,1,0,0,0,0,14,31934
2151,588712,17943,0.125294,0.026449,-0.000241,-0.022211,-0.081749,-0.028564,-0.033777,0.011203,...,1,1,1,1,1,1,1,1,48,121902
44586,670600,17943,0.327109,0.123322,-0.050781,-0.068396,-0.167216,-0.048022,-0.064586,0.075355,...,1,1,1,1,1,1,1,1,34,592790


In [18]:
def deepwalk(df, uid, aid, L):
    dic = {}
    for k, v in df[[uid, aid]].values:
        if k not in dic:
            dic["uid_"+str(k)] = ["aid_"+str(v)]
        else:
            dic["uid_"+str(k)].append("aid_"+str(v))
        if v not in dic:
            dic["aid_"+str(v)] = ["uid_"+str(k)]
        else:
            dic["aid_"+str(v)].append("uid_"+str(k))
    
    sentences = []
    path_length = 10
    
    for i in range(5):
        for k in dic:
            sentence = [k]
            while len(sentence) != path_length:
                key = np.random.choice(dic[sentence[-1]])
                if len(sentence) >= 2 and key == sentence[-2]:
                    break
                sentence.append(key)
            sentences.append(sentence)
    
    model = Word2Vec(sentences, size=L, window=3, min_count=1, workers=10, iter=50)
    embedding = [[v] + list(model.wv["aid_"+str(v)]) for v in df[aid].unique()]
    columns = [aid] + ["%s_deepwalk_%d" % (aid, i) for i in range(L)]
    embed_df = pd.DataFrame(embedding, columns=columns)
    return embed_df, sentences

In [24]:
embed_df, sentences = deepwalk(df, "uid", "aid", 64)

In [27]:
embed_df.head()

,aid,aid_deepwalk_0,aid_deepwalk_1,aid_deepwalk_2,aid_deepwalk_3,aid_deepwalk_4,aid_deepwalk_5,aid_deepwalk_6,aid_deepwalk_7,aid_deepwalk_8,...,aid_deepwalk_54,aid_deepwalk_55,aid_deepwalk_56,aid_deepwalk_57,aid_deepwalk_58,aid_deepwalk_59,aid_deepwalk_60,aid_deepwalk_61,aid_deepwalk_62,aid_deepwalk_63
0,357876,0.633634,-0.341933,0.564209,-0.636698,1.089705,-1.448567,-0.105144,-0.158042,0.308099,...,-0.622365,1.370297,0.340295,0.588869,0.026966,-1.137348,0.128776,0.246614,0.024890,-0.456423
1,561658,-0.528444,0.128493,0.210712,-0.178420,0.128783,0.153999,-0.128505,-0.338816,0.594167,...,-0.625296,0.276972,0.600817,0.268649,-0.493468,-0.429911,-0.317110,-0.116697,0.371480,-0.324162
2,369559,-1.874767,1.360924,0.127445,-0.412853,3.713765,0.350264,-0.217665,2.963290,1.036064,...,-2.045743,0.654691,0.796730,1.048428,0.487945,-0.910348,-0.614299,-1.813997,-0.677300,-1.472522
3,247140,-0.020025,0.528048,0.383748,-0.263099,0.349827,0.192379,-0.231892,0.316442,0.070833,...,-0.319712,0.333239,-0.187063,0.102425,0.308581,-0.202362,0.183862,0.198653,-0.014661,0.368015
4,559211,-0.039206,0.070238,-0.023012,-0.117294,0.301481,0.102307,-0.148982,-0.388983,0.126792,...,-0.183254,0.096638,0.038552,0.101032,-0.029148,0.271945,-0.021971,0.057125,0.589131,-0.403249


In [29]:
# new_df = df.merge(embed_df, how="inner", on="aid")
new_df.shape, embed_df.shape, df.shape

((169590, 334), (28401, 65), (169590, 270))

In [61]:
new_df.aid.nunique()

28401

In [36]:
new_df = new_df.drop("uid", axis=1)

In [53]:
new_df.iloc[:, :-64].head()

,aid,request_day,aid_embed_0,aid_embed_1,aid_embed_2,aid_embed_3,aid_embed_4,aid_embed_5,aid_embed_6,aid_embed_7,...,periods_on_39,periods_on_40,periods_on_41,periods_on_42,periods_on_43,periods_on_44,periods_on_45,periods_on_46,periods_on_47,periods_cont
0,357876,17943,0.255362,0.184418,-0.069231,-0.150381,-0.090089,-0.098903,-0.070554,0.014766,...,0,0,0,0,0,0,0,1,1,2
1,357876,17944,0.255362,0.184418,-0.069231,-0.150381,-0.090089,-0.098903,-0.070554,0.014766,...,0,0,0,0,0,0,0,1,1,2
2,357876,17945,0.255362,0.184418,-0.069231,-0.150381,-0.090089,-0.098903,-0.070554,0.014766,...,0,0,0,0,0,0,0,1,1,2
3,357876,17946,0.255362,0.184418,-0.069231,-0.150381,-0.090089,-0.098903,-0.070554,0.014766,...,0,0,0,0,0,0,0,1,1,2
4,357876,17947,0.255362,0.184418,-0.069231,-0.150381,-0.090089,-0.098903,-0.070554,0.014766,...,0,0,0,0,0,0,0,1,1,2


In [54]:
test_df = new_df[new_df.request_day==17974].iloc[:, 1:].iloc[:, :-64]
train_df = new_df[new_df.request_day != 17974].iloc[:, 1:].iloc[:, :-64]

# val_df = train_df[train_df.request_day == 17973].iloc[:, 1:]
# train_df = train_df[train_df.request_day != 17973].iloc[:, 1:]

In [55]:
Y_train, Y_test = train_df["imp"], test_df["imp"]
X_train = train_df.drop("imp", axis=1)
X_test = test_df.drop("imp", axis=1)

In [45]:
# Y_train, Y_val, Y_test = train_df["imp"], val_df["imp"].values, test_df["imp"]
# X_train = train_df.drop("imp", axis=1)
# X_val = val_df.drop("imp", axis=1)
# X_test = test_df.drop("imp", axis=1)

In [56]:
list(train_df.columns)

['request_day',
 'aid_embed_0',
 'aid_embed_1',
 'aid_embed_2',
 'aid_embed_3',
 'aid_embed_4',
 'aid_embed_5',
 'aid_embed_6',
 'aid_embed_7',
 'aid_embed_8',
 'aid_embed_9',
 'aid_embed_10',
 'aid_embed_11',
 'aid_embed_12',
 'aid_embed_13',
 'aid_embed_14',
 'aid_embed_15',
 'aid_embed_16',
 'aid_embed_17',
 'aid_embed_18',
 'aid_embed_19',
 'aid_embed_20',
 'aid_embed_21',
 'aid_embed_22',
 'aid_embed_23',
 'aid_embed_24',
 'aid_embed_25',
 'aid_embed_26',
 'aid_embed_27',
 'aid_embed_28',
 'aid_embed_29',
 'aid_embed_30',
 'aid_embed_31',
 'aid_embed_32',
 'aid_embed_33',
 'aid_embed_34',
 'aid_embed_35',
 'aid_embed_36',
 'aid_embed_37',
 'aid_embed_38',
 'aid_embed_39',
 'aid_embed_40',
 'aid_embed_41',
 'aid_embed_42',
 'aid_embed_43',
 'aid_embed_44',
 'aid_embed_45',
 'aid_embed_46',
 'aid_embed_47',
 'aid_embed_48',
 'aid_embed_49',
 'aid_embed_50',
 'aid_embed_51',
 'aid_embed_52',
 'aid_embed_53',
 'aid_embed_54',
 'aid_embed_55',
 'aid_embed_56',
 'aid_embed_57',
 'aid_em

In [57]:
categorical_features = ["DayOfWeek", "good_type", "ad_type_id", "ad_size"]
train_data = lgb.Dataset(X_train, label=Y_train, categorical_feature=categorical_features, free_raw_data=False)
# val_data = lgb.Dataset(X_val, label=Y_val, reference=train_data, categorical_feature=categorical_features, free_raw_data=False)
test_data = lgb.Dataset(X_test, label=Y_test, reference=train_data, categorical_feature=categorical_features, free_raw_data=False)



In [58]:
def my_smape(y_pred, dtrain):
    y_true = dtrain.get_label()
    smape = 2 * np.mean(np.abs(y_pred - y_true)/(y_pred + y_true))
    return "smape", smape, False

In [59]:
params = {"num_leaves": 100, 
          "max_depth": 15,
          "min_data_in_leaf": 30,
          "objective": "mae",
          "metric": "mae",
          "random_state": 2019,
          "learning_rate":0.01,
          "lambda_l2": 0.1,
          "bagging_fraction": 0.9,
          "feature_fraction": 0.9,
          "save_binary": True
        }

num_round = 10000
bst = lgb.train(params, train_data, num_round, valid_sets=[train_data, test_data], feval=my_smape, early_stopping_rounds=100, verbose_eval=True)


/home/ubuntu/workspaces/py3/lib/python3.6/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[1]	training's l1: 7.37251	training's smape: 0.878278	valid_1's l1: 8.90906	valid_1's smape: 0.901257
Training until validation scores don't improve for 100 rounds
[2]	training's l1: 7.33756	training's smape: 0.873732	valid_1's l1: 8.8616	valid_1's smape: 0.895271
[3]	training's l1: 7.30091	training's smape: 0.869056	valid_1's l1: 8.81182	valid_1's smape: 0.889295
[4]	training's l1: 7.26584	training's smape: 0.864527	valid_1's l1: 8.76465	valid_1's smape: 0.883523
[5]	training's l1: 7.23009	training's smape: 0.860032	valid_1's l1: 8.71584	valid_1's smape: 0.877751
[6]	training's l1: 7.19671	training's smape: 0.855779	valid_1's l1: 8.67029	valid_1's smape: 0.872235
[7]	training's l1: 7.16391	training's smape: 0.85159	valid_1's l1: 8.62661	valid_1's smape: 0.866948
[8]	training's l1: 7.13143	training's smape: 0.847477	valid_1's l1: 8.5834	valid_1's smape: 0.861775
[9]	training's l1: 7.09916	training's smape: 0.84348	valid_1's l1: 8.53993	valid_1's smape: 0.856722
[10]	training's l1: 7.06

[81]	training's l1: 5.63941	training's smape: 0.678949	valid_1's l1: 6.54007	valid_1's smape: 0.665431
[82]	training's l1: 5.62757	training's smape: 0.677646	valid_1's l1: 6.52425	valid_1's smape: 0.664059
[83]	training's l1: 5.61609	training's smape: 0.676382	valid_1's l1: 6.5085	valid_1's smape: 0.662752
[84]	training's l1: 5.60441	training's smape: 0.675105	valid_1's l1: 6.49217	valid_1's smape: 0.661411
[85]	training's l1: 5.59311	training's smape: 0.673865	valid_1's l1: 6.477	valid_1's smape: 0.660153
[86]	training's l1: 5.58222	training's smape: 0.672605	valid_1's l1: 6.46298	valid_1's smape: 0.658933
[87]	training's l1: 5.57134	training's smape: 0.671361	valid_1's l1: 6.44847	valid_1's smape: 0.657672
[88]	training's l1: 5.56066	training's smape: 0.670145	valid_1's l1: 6.43426	valid_1's smape: 0.656481
[89]	training's l1: 5.55021	training's smape: 0.668931	valid_1's l1: 6.42069	valid_1's smape: 0.655293
[90]	training's l1: 5.54111	training's smape: 0.667799	valid_1's l1: 6.40901

[163]	training's l1: 5.06256	training's smape: 0.612391	valid_1's l1: 5.7664	valid_1's smape: 0.599967
[164]	training's l1: 5.05884	training's smape: 0.611945	valid_1's l1: 5.76137	valid_1's smape: 0.599515
[165]	training's l1: 5.05544	training's smape: 0.611473	valid_1's l1: 5.75705	valid_1's smape: 0.599073
[166]	training's l1: 5.0515	training's smape: 0.611004	valid_1's l1: 5.75298	valid_1's smape: 0.598699
[167]	training's l1: 5.048	training's smape: 0.610575	valid_1's l1: 5.7489	valid_1's smape: 0.598334
[168]	training's l1: 5.0445	training's smape: 0.61014	valid_1's l1: 5.74454	valid_1's smape: 0.597977
[169]	training's l1: 5.04098	training's smape: 0.609705	valid_1's l1: 5.74034	valid_1's smape: 0.597631
[170]	training's l1: 5.0375	training's smape: 0.609263	valid_1's l1: 5.73622	valid_1's smape: 0.59729
[171]	training's l1: 5.03407	training's smape: 0.608857	valid_1's l1: 5.73237	valid_1's smape: 0.596935
[172]	training's l1: 5.03065	training's smape: 0.608428	valid_1's l1: 5.7

[244]	training's l1: 4.86902	training's smape: 0.585706	valid_1's l1: 5.55791	valid_1's smape: 0.578942
[245]	training's l1: 4.86746	training's smape: 0.585471	valid_1's l1: 5.55653	valid_1's smape: 0.578772
[246]	training's l1: 4.86589	training's smape: 0.58524	valid_1's l1: 5.55503	valid_1's smape: 0.578607
[247]	training's l1: 4.86445	training's smape: 0.585007	valid_1's l1: 5.55397	valid_1's smape: 0.578484
[248]	training's l1: 4.863	training's smape: 0.58478	valid_1's l1: 5.5523	valid_1's smape: 0.578311
[249]	training's l1: 4.8615	training's smape: 0.584561	valid_1's l1: 5.5507	valid_1's smape: 0.578153
[250]	training's l1: 4.86003	training's smape: 0.584336	valid_1's l1: 5.54912	valid_1's smape: 0.57799
[251]	training's l1: 4.85858	training's smape: 0.58412	valid_1's l1: 5.54764	valid_1's smape: 0.577832
[252]	training's l1: 4.85728	training's smape: 0.583892	valid_1's l1: 5.5464	valid_1's smape: 0.577671
[253]	training's l1: 4.85594	training's smape: 0.583677	valid_1's l1: 5.54

...

[405]	training's l1: 4.72084	training's smape: 0.561019	valid_1's l1: 5.45839	valid_1's smape: 0.565379
[406]	training's l1: 4.72041	training's smape: 0.560928	valid_1's l1: 5.45825	valid_1's smape: 0.565355
[407]	training's l1: 4.71983	training's smape: 0.560829	valid_1's l1: 5.45781	valid_1's smape: 0.5653
[408]	training's l1: 4.7194	training's smape: 0.560736	valid_1's l1: 5.45778	valid_1's smape: 0.565276
[409]	training's l1: 4.71882	training's smape: 0.560634	valid_1's l1: 5.45762	valid_1's smape: 0.56525
[410]	training's l1: 4.71824	training's smape: 0.560555	valid_1's l1: 5.45707	valid_1's smape: 0.565178
[411]	training's l1: 4.71766	training's smape: 0.560459	valid_1's l1: 5.4568	valid_1's smape: 0.565152
[412]	training's l1: 4.71714	training's smape: 0.560366	valid_1's l1: 5.4565	valid_1's smape: 0.565099
[413]	training's l1: 4.71651	training's smape: 0.560272	valid_1's l1: 5.4564	valid_1's smape: 0.565061
[414]	training's l1: 4.71586	training's smape: 0.560176	valid_1's l1: 5

[486]	training's l1: 4.67761	training's smape: 0.554117	valid_1's l1: 5.4457	valid_1's smape: 0.562893
[487]	training's l1: 4.67719	training's smape: 0.554046	valid_1's l1: 5.44577	valid_1's smape: 0.56289
[488]	training's l1: 4.67676	training's smape: 0.553976	valid_1's l1: 5.44564	valid_1's smape: 0.562861
[489]	training's l1: 4.67635	training's smape: 0.553907	valid_1's l1: 5.44572	valid_1's smape: 0.562843
[490]	training's l1: 4.67609	training's smape: 0.553848	valid_1's l1: 5.44572	valid_1's smape: 0.56283
[491]	training's l1: 4.67572	training's smape: 0.553784	valid_1's l1: 5.44585	valid_1's smape: 0.562804
[492]	training's l1: 4.67512	training's smape: 0.553708	valid_1's l1: 5.44577	valid_1's smape: 0.562784
[493]	training's l1: 4.67485	training's smape: 0.553649	valid_1's l1: 5.44578	valid_1's smape: 0.56278
[494]	training's l1: 4.67449	training's smape: 0.553584	valid_1's l1: 5.44572	valid_1's smape: 0.562747
[495]	training's l1: 4.67396	training's smape: 0.553512	valid_1's l1

[566]	training's l1: 4.64345	training's smape: 0.548909	valid_1's l1: 5.43925	valid_1's smape: 0.561288
[567]	training's l1: 4.64312	training's smape: 0.54886	valid_1's l1: 5.43901	valid_1's smape: 0.561276
[568]	training's l1: 4.64271	training's smape: 0.548807	valid_1's l1: 5.43887	valid_1's smape: 0.561273
[569]	training's l1: 4.6423	training's smape: 0.548746	valid_1's l1: 5.43872	valid_1's smape: 0.56126
[570]	training's l1: 4.6419	training's smape: 0.548689	valid_1's l1: 5.43867	valid_1's smape: 0.561263
[571]	training's l1: 4.64143	training's smape: 0.548628	valid_1's l1: 5.43853	valid_1's smape: 0.561262
[572]	training's l1: 4.64103	training's smape: 0.548573	valid_1's l1: 5.43841	valid_1's smape: 0.561269
[573]	training's l1: 4.64067	training's smape: 0.548523	valid_1's l1: 5.43814	valid_1's smape: 0.561251
[574]	training's l1: 4.64028	training's smape: 0.548469	valid_1's l1: 5.43816	valid_1's smape: 0.561254
[575]	training's l1: 4.64002	training's smape: 0.548418	valid_1's l1

[647]	training's l1: 4.61923	training's smape: 0.544903	valid_1's l1: 5.43393	valid_1's smape: 0.559963
[648]	training's l1: 4.61897	training's smape: 0.544854	valid_1's l1: 5.43375	valid_1's smape: 0.559925
[649]	training's l1: 4.61869	training's smape: 0.544813	valid_1's l1: 5.43354	valid_1's smape: 0.559905
[650]	training's l1: 4.61839	training's smape: 0.544758	valid_1's l1: 5.43348	valid_1's smape: 0.55991
[651]	training's l1: 4.61831	training's smape: 0.544726	valid_1's l1: 5.4335	valid_1's smape: 0.559919
[652]	training's l1: 4.61807	training's smape: 0.544674	valid_1's l1: 5.43344	valid_1's smape: 0.559902
[653]	training's l1: 4.61787	training's smape: 0.544634	valid_1's l1: 5.43355	valid_1's smape: 0.559842
[654]	training's l1: 4.61762	training's smape: 0.544584	valid_1's l1: 5.43352	valid_1's smape: 0.559829
[655]	training's l1: 4.61752	training's smape: 0.544549	valid_1's l1: 5.43348	valid_1's smape: 0.559818
[656]	training's l1: 4.61741	training's smape: 0.544519	valid_1's 

[729]	training's l1: 4.59809	training's smape: 0.541105	valid_1's l1: 5.44828	valid_1's smape: 0.561132
[730]	training's l1: 4.59784	training's smape: 0.541052	valid_1's l1: 5.4482	valid_1's smape: 0.561112
[731]	training's l1: 4.59753	training's smape: 0.540994	valid_1's l1: 5.44814	valid_1's smape: 0.561107
[732]	training's l1: 4.59719	training's smape: 0.540942	valid_1's l1: 5.44852	valid_1's smape: 0.561208
[733]	training's l1: 4.59694	training's smape: 0.540902	valid_1's l1: 5.4494	valid_1's smape: 0.561379
[734]	training's l1: 4.5967	training's smape: 0.540863	valid_1's l1: 5.44987	valid_1's smape: 0.561524
[735]	training's l1: 4.59649	training's smape: 0.540822	valid_1's l1: 5.45134	valid_1's smape: 0.561685
[736]	training's l1: 4.59639	training's smape: 0.540796	valid_1's l1: 5.45141	valid_1's smape: 0.56175
[737]	training's l1: 4.59625	training's smape: 0.540764	valid_1's l1: 5.45204	valid_1's smape: 0.561934
[738]	training's l1: 4.59605	training's smape: 0.540725	valid_1's l1

In [43]:
test_data = lgb.Dataset(X_test, Y_test, categorical_feature=categorical_features, free_raw_data=False)
y_pred = bst.predict(X_test, num_iteration=bst.best_iteration)
my_smape(y_pred, test_data)

('smape', 0.5704297197070947, False)

### 不过滤异常值

In [28]:
# 同样参数设置，不过滤异常值
# val [100]	valid_0's l1: 5.77628	valid_0's smape: 0.575621
# test 0.5674792241044383
params = {"num_leaves": 32, 
          "max_depth": 8,
          "min_data_in_leaf": 100,
          "objective": "mae",
          "metric": "mae",
          "random_state": 2019,
          "learning_rate":0.1,
          "lambda_l2": 0.1,
          "bagging_fraction": 0.9,
          "feature_fraction":0.8,
          "save_binary": True
        }

num_round = 100
bst = lgb.train(params, train_data, num_round, valid_sets=[train_data, val_data], feval=my_smape, early_stopping_rounds=5, verbose_eval=True)


/home/ubuntu/workspaces/py3/lib/python3.6/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[1]	training's l1: 28.3452	training's smape: 0.877118	valid_1's l1: 30.9372	valid_1's smape: 0.898583
Training until validation scores don't improve for 5 rounds
[2]	training's l1: 27.7468	training's smape: 0.840741	valid_1's l1: 30.2353	valid_1's smape: 0.8537
[3]	training's l1: 26.5647	training's smape: 0.80407	valid_1's l1: 28.7428	valid_1's smape: 0.807206
[4]	training's l1: 26.336	training's smape: 0.785084	valid_1's l1: 28.4699	valid_1's smape: 0.787714
[5]	training's l1: 25.3687	training's smape: 0.759092	valid_1's l1: 27.2444	valid_1's smape: 0.755801
[6]	training's l1: 24.5638	training's smape: 0.738541	valid_1's l1: 26.2858	valid_1's smape: 0.732865
[7]	training's l1: 24.3745	training's smape: 0.725575	valid_1's l1: 26.1235	valid_1's smape: 0.722167
[8]	training's l1: 24.2408	training's smape: 0.714246	valid_1's l1: 25.9864	valid_1's smape: 0.712843
[9]	training's l1: 24.0734	training's smape: 0.703163	valid_1's l1: 25.8101	valid_1's smape: 0.703617
[10]	training's l1: 23.923

In [29]:
test_data = lgb.Dataset(X_test, Y_test, categorical_feature=categorical_features, free_raw_data=False)
y_pred = bst.predict(X_test, num_iteration=bst.best_iteration)
my_smape(y_pred, test_data)

('smape', 0.5936661647536137, False)

In [ ]:
df.imp.value_counts()